# 3차모델 : 회귀분석 
## 시간데이터를 범주형 변수로 변형시켜서 시간 개념을 넣어보기로 하였고
## 모든 시간 개념을 다 가져갈 수는 없겠지만, 공장 특성상
## 최근값에 영향을 매우 많이 받기에 최근 일자들 개념만 가져가도 될 것 같았어요
## 데이터는 토요일 기준으로 짤라서 직전 주까지의 데이터를 가져오고
## 모델을 돌리는 시점의 다음주 출고량부터 4주치를 예측하게됩니다
## 제품별 데이터들 (유형 , 단위 , 규격) 같은 컬럼 가져가려고 했는데
## 코드 자체가 제품 하나씩 돌리는 거다 보니까 제품마다 다른값을 가져가봐야
## 의미가 없다라고 판단이 들어서 가지고 가지 않았습니다
## 각 제품마다 선택적 제거법으로 변수선택을 하였고, 
## over/underfitting 을 판단하기가 애매해서 모든 데이터를 다 학습시켰습니다

In [1]:
import pandas as pd
import pymssql
import numpy as np
import itertools
from datetime import datetime 
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import time
import itertools
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics

In [2]:
connet = pymssql.connect(server = '121.134.7.213',user='sa',password='twinet3015!', database='SEIERP_IYN_BAK')

sql_query = '''
declare	 @CD_CORP         NVARCHAR(2)

  , @DT_DEL_F        NVARCHAR(8)

  , @DT_DEL_T        NVARCHAR(8)



set @CD_CORP	=	'01'

set	@DT_DEL_F	=	'20200101'

set @DT_DEL_T	=	(SELECT CONVERT(CHAR(8), GETDATE(), 112))



    SET NOCOUNT ON

	SET ANSI_WARNINGS OFF

	SET ARITHIGNORE ON

	SET ARITHABORT OFF



;With DeliveryCross AS (

 select a.CD_EMP

 , dbo.GetFormat_Day(B.DT_COMP,'-') as DT_COMP

 , B.NO_SMDL, B.NO_SMDLSUB

 , B.QT_SMDL, B.UP_PRICE, B.AM_PRICE, B.AM_VAT, B.AM_TOT, B.TX_NOTE, B.NO_SMOR

 , c.NM_CUST as NM_DELCUST

 , C1.NM_CUST

 , k.NM_USER, r.NM_WAREHOUSE

 , i.CD_ITEM, i.NM_ITEM, i.TX_SPEC, n.NM_TYPEDTL as NM_UNIT

 , DT2.NM_TYPEDTL		NM_WONGA

 FROM SMDL_DELIVERY_H a

 Left outer Join SMDL_DELIVERY_D as b on a.CD_CORP=b.CD_CORP and a.NO_SMDL=b.NO_SMDL

 Left outer Join BSCT_CUST c on b.CD_CORP=c.CD_CORP and b.CD_DELCUST=c.CD_CUST

 Left outer Join BSCT_CUST c1 on b.CD_CORP=c1.CD_CORP and b.CD_CUST=c1.CD_CUST

 Left outer Join BSIN_USER k on a.CD_CORP=k.CD_CORP and a.CD_EMP=k.CD_EMP

 Left outer Join STIT_WAREHOUSE r on b.CD_CORP=r.CD_CORP and b.CD_WAREHOUSE=r.CD_WAREHOUSE		--창고코드

 Left outer Join BSIT_ITEM i on b.CD_CORP=i.CD_CORP and b.CD_SYSITEM=i.CD_SYSITEM				--	품목코드

 Left outer Join (select * from BSIN_CODE_DTL) n on i.CD_CORP=n.CD_CORP and i.CD_UNIT=n.CD_TYPEDTL And n.CD_TYPE='MI001'

 Left outer Join (Select * From BSIN_CODE_DTL) t1 on b.CD_CORP=t1.CD_CORP and b.CD_SELLTYPE=t1.CD_TYPEDTL and t1.CD_TYPE='SM004'

 Left outer Join FROP_STORE m On c.CD_CORP = m.CD_CORP AND c.CD_STORECODE = m.CD_STORE			--매장코드

  LEFT OUTER JOIN BSIN_CODE_DTL DT2  

     ON I.CD_CORP = DT2.CD_CORP  

    AND DT2.CD_TYPE = 'MI032'  

    AND I.CD_WONGA = DT2.CD_TYPEDTL  

 WHERE 1=1

       AND A.CD_CORP    = @CD_CORP

	   AND a.DT_COMP	>=	@DT_DEL_F 

	   AND a.DT_COMP	<=	@DT_DEL_T

       AND ISNULL(B.CK_BUY, 0) <> 9			--공장출고만

 )

 SELECT 1 div_out, NM_WONGA,	DT_COMP, CD_ITEM, NM_ITEM , TX_SPEC , NM_UNIT

 , sum(QT_SMDL) QT_SMDL, ((sum(AM_PRICE)) / sum(QT_SMDL)) UP_PRICE --avg(UP_PRICE) UP_PRICE

 , sum(AM_VAT) AM_VAT, sum(AM_PRICE) AM_PRICE, sum(AM_PRICE)+sum(AM_VAT) AM_TOT

 , NM_CUST

 , NM_DELCUST

 

 FROM DeliveryCross

 GROUP BY NM_WONGA,	CD_ITEM, NM_ITEM  , TX_SPEC , NM_UNIT, DT_COMP

 , NM_CUST

 , NM_DELCUST

 UNION ALL

 SELECT 2 div_out, '', ''	NM_WONGA,	'총계' itemcode1, '' itemname , '' ,''

 , sum(QT_SMDL) QT_SMDL, ((sum(AM_PRICE)) / sum(QT_SMDL)) UP_PRICE  --, avg(UP_PRICE) UP_PRICE

 , sum(AM_VAT) AM_VAT, sum(AM_PRICE) AM_PRICE, sum(AM_PRICE)+sum(AM_VAT) AM_TOT

 , ''	NM_CUST

 , ''	NM_DELCUST

 FROM DeliveryCross

 order by div_out, CD_ITEM
 '''
row_data = pd.read_sql(sql=sql_query,con=connet)


C:\Users\ioi\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
item_list = ['한촌탕고기(뒷덩이)',
 '한촌수육양지',
 '한촌설렁탕육수(2.5kg)',
 '삶은소건(2kg)',
 '사골추출액-1202(냉장)',
 '한촌매운돼지갈비소스',
 '한촌양념불고기',
 '혼합김치 18Kg',
'곰탕육수(구.나주곰탕육수)',
            '한촌수육머리고기(구,곰탕용)']
data = row_data[row_data['NM_ITEM'].isin(item_list)]

data.drop(['div_out','CD_ITEM','AM_PRICE','NM_CUST','NM_DELCUST','AM_VAT','AM_TOT' ],axis=1 , inplace = True)
data.rename(columns = {'NM_WONGA' : '유형','DT_COMP' : '일자','NM_ITEM' : '품명',
                      'NM_UNIT' : '규격', 'QT_SMDL' : '출고수량','UP_PRICE' : '단가'}, inplace=True)

pd.set_option('mode.chained_assignment',  None)

data1 = data.groupby(['일자','품명','유형','TX_SPEC','규격']).agg({'출고수량':'sum',
                                                  '단가':'mean'})
data1 = data1.reset_index()

data1 = data1[data1['출고수량'] >0]

data1 = data1.astype({'출고수량':'int','단가':'int'})

data1 = data1.astype({'TX_SPEC':'string'})
data1['TX_SPEC'].iloc[0].split('g')[1].split('1')[0]

data1 = data1.astype({'TX_SPEC':'string'})
data1['중량(kg)'] = data1['TX_SPEC'].str.split('×').str[0].str.split('k').str[0]
data1.drop(['TX_SPEC'],axis=1,inplace=True)

data1['일자'] = pd.to_datetime(data1['일자'])
data1


C:\Users\ioi\AppData\Local\Temp\ipykernel_21572\3037783778.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['div_out','CD_ITEM','AM_PRICE','NM_CUST','NM_DELCUST','AM_VAT','AM_TOT' ],axis=1 , inplace = True)
C:\Users\ioi\AppData\Local\Temp\ipykernel_21572\3037783778.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = {'NM_WONGA' : '유형','DT_COMP' : '일자','NM_ITEM' : '품명',


,일자,품명,유형,규격,출고수량,단가,중량(kg)
0,2020-01-01,사골추출액-1202(냉장),추출가공,PACK,212,5981,2.5
1,2020-01-01,삶은소건(2kg),추출가공,PACK,6,44022,2
2,2020-01-01,한촌매운돼지갈비소스,소스,PACK,18,10910,3
3,2020-01-01,한촌설렁탕육수(2.5kg),추출가공,PACK,1072,8613,2.5
4,2020-01-01,"한촌수육머리고기(구,곰탕용)",추출가공,PACK,72,28135,1.5
...,...,...,...,...,...,...,...
9418,2023-02-27,"한촌수육머리고기(구,곰탕용)",추출가공,PACK,48,42320,1.5
9419,2023-02-27,한촌수육양지,추출가공,PACK,26,111840,3
9420,2023-02-27,한촌양념불고기,양념육,PACK,52,46350,4
9421,2023-02-27,한촌탕고기(뒷덩이),추출가공,PACK,16,26000,0.5


### forward / backward / stepwise : 변수 선택법 중  전진 , 후진 , 선택적 제거법 

In [ ]:
    
def processSubset(X,y,feature_set) :
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    AIC = regr.aic
    return {'model' : regr, 'AIC': AIC}
def forward(X, y, predictors):
    # 데이터 변수들이 미리정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['const']) if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y= y, feature_set=predictors+[p]+['const']))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)

    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[0] )
    return best_model

def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=["AIC", "model"])
    tic = time.time()
    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0~9 -> 1~10
    for i in range(1, len(X.columns.difference(['const'])) + 1):
        Forward_result = forward(X=X,y=y,predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapsed time:", (toc - tic), "seconds.")

    return(Fmodels['model'][len(Fmodels['model'])])

def backward(X,y,predictors):
    tic = time.time()
    results = []
    # 데이터 변수들이 미리정의된 predictors 조합 확인
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(X=X, y= y,feature_set=list(combo)+['const']))
    models = pd.DataFrame(results)
    # 가장 낮은 AIC를 가진 모델을 선택
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors) - 1, "predictors in",
          (toc - tic))
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[0] )
    return best_model
def backward_model(X, y):
    Bmodels = pd.DataFrame(columns=["AIC", "model"], index = range(1,len(X.columns)))
    tic = time.time()
    predictors = X.columns.difference(['const'])
    Bmodel_before = processSubset(X,y,predictors)['AIC']
    while (len(predictors) > 1):
        Backward_result = backward(X=x_train, y= y_train, predictors = predictors)
        if Backward_result['AIC'] > Bmodel_before:
            break
        Bmodels.loc[len(predictors) - 1] = Backward_result
        predictors = Bmodels.loc[len(predictors) - 1]["model"].model.exog_names
        Bmodel_before = Backward_result['AIC']
        predictors = [ k for k in predictors if k != 'const']

    toc = time.time()
    print("Total elapsed time:", (toc - tic), "seconds.")
    return (Bmodels['model'].dropna().iloc[0])

def Stepwise_model(X,y):
    Stepmodels = pd.DataFrame(columns=["AIC", "model"])
    tic = time.time()
    predictors = []
    Smodel_before = processSubset(X,y,predictors+['const'])['AIC']
    # 변수 1~10개 : 0~9 -> 1~10
    for i in range(1, len(X.columns.difference(['const'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors) # constant added
        print('forward')
        Stepmodels.loc[i] = Forward_result
        predictors = Stepmodels.loc[i]["model"].model.exog_names
        predictors = [ k for k in predictors if k != 'const']
        Backward_result = backward(X=X, y=y, predictors=predictors)
        if Backward_result['AIC']< Forward_result['AIC']:
            Stepmodels.loc[i] = Backward_result
            predictors = Stepmodels.loc[i]["model"].model.exog_names
            Smodel_before = Stepmodels.loc[i]["AIC"]
            predictors = [ k for k in predictors if k != 'const']
            print('backward')
        if Stepmodels.loc[i]['AIC']> Smodel_before:
            break
        else:
            Smodel_before = Stepmodels.loc[i]["AIC"]
    toc = time.time()
    print("Total elapsed time:", (toc - tic), "seconds.")
    return (Stepmodels['model'][len(Stepmodels['model'])])

### 시간데이터를 범주형변수로 바꾸는데 최근 8주치의 출고량이 있어야하므로, 
### 데이터 시작지점부터 8주간의 데이터는 날라감
### ap_data : 앞으로 예측할 기간의 데이터셋 
### merge_data : ap_data의 가장 가까운 일자 한행 + data6(범주형변수로 돌리기전)
### 즉, 예측할 한행의 독립변수값을 집어넣기위한 변수 , 이후 출고량 예측
### 한행 예측 후에, 다시 ap_data의 두번째 가까운 일자 행 + merge 로 concat 후 또 예측

In [6]:
final_result = []
rsquared=[]
rmse1=[]

for n in range(len(item_list)) :
    data2_1 = data1[data1['품명']==item_list[n]]
    data2_1 = data2_1[['일자','출고수량','단가']]
    data2_1 = data2_1.reset_index()
    while (data2_1['일자'].iloc[len(data2_1)-1].weekday() != 5 ):
        data2_1 = data2_1.drop(len(data2_1)-1,axis=0)
    data2_1.set_index(data2_1['일자'],inplace=True)
    data2_2 = data2_1.drop(['일자'],axis=1)
    week_df1 = data2_2['출고수량'].resample('W-Sat').sum().reset_index()
    week_df2 = data2_2['단가'].resample('W-Sat').mean().reset_index()
    data2 = pd.merge(week_df1,week_df2)
    
    data2['1주전출고량'] = np.nan
    data2['2주전출고량'] = np.nan
    data2['3주전출고량'] = np.nan
    data2['4주전출고량'] = np.nan
    data2['5주전출고량'] = np.nan
    data2['6주전출고량'] = np.nan
    data2['7주전출고량'] = np.nan
    data2['8주전출고량'] = np.nan
    data2['4주평균'] = np.nan
    data2['8주평균'] = np.nan
    data2['4주간최대'] = np.nan
    data2['4주간최소'] = np.nan
    last_data = data2[['일자','출고수량']]
    
    for i in range(len(data2)):
        abc = last_data[last_data['일자'] < data2['일자'].iloc[i] ].tail(8)
        if len(abc) > 7 :
            data2['8주전출고량'].iloc[i] = abc['출고수량'].iloc[0]
            data2['7주전출고량'].iloc[i] = abc['출고수량'].iloc[1]
            data2['6주전출고량'].iloc[i] = abc['출고수량'].iloc[2]
            data2['5주전출고량'].iloc[i] = abc['출고수량'].iloc[3]
            data2['4주전출고량'].iloc[i] = abc['출고수량'].iloc[4]
            data2['3주전출고량'].iloc[i] = abc['출고수량'].iloc[5]
            data2['2주전출고량'].iloc[i] = abc['출고수량'].iloc[6]
            data2['1주전출고량'].iloc[i] = abc['출고수량'].iloc[7]       
            data2['4주평균'].iloc[i] = abc['출고수량'].tail(4).mean()
            data2['8주평균'].iloc[i] = abc['출고수량'].mean()
            data2['4주간최대'].iloc[i] = abc['출고수량'].tail(4).max()
            data2['4주간최소'].iloc[i] = abc['출고수량'].tail(4).min()


        else :
            data2['8주전출고량'].iloc[i] = None
            data2['7주전출고량'].iloc[i] =None
            data2['6주전출고량'].iloc[i] = None
            data2['5주전출고량'].iloc[i] = None
            data2['4주전출고량'].iloc[i] =None
            data2['3주전출고량'].iloc[i] = None
            data2['2주전출고량'].iloc[i] = None
            data2['1주전출고량'].iloc[i] = None
            data2['4주평균'].iloc[i] = None
            data2['8주평균'].iloc[i] = None
            data2['4주간최대'].iloc[i] = None
            data2['4주간최소'].iloc[i] = None
            
    data6 = data2.copy()
    data6['주차'] = data6['일자'].dt.week
    data6['월'] = data6['일자'].dt.month
    data6['년'] = data6['일자'].dt.year
    data6 = data6.astype({'주차':'object',
                       '월':'object','년':'object'})
    
        
    data6['1주전변동'] = np.nan
    data6['2주전변동'] = np.nan
    data6['3주전변동'] = np.nan
    data6['4주전변동'] = np.nan

    data6['1주전변동'] = data6['2주전출고량'] - data6['1주전출고량']
    data6['2주전변동'] = data6['3주전출고량'] - data6['2주전출고량']
    data6['3주전변동'] = data6['4주전출고량'] - data6['3주전출고량']
    data6['4주전변동'] = data6['5주전출고량'] - data6['4주전출고량']
    
    data6.dropna(axis=0,inplace=True)
    data7 = pd.get_dummies(data6,columns=['주차','월','년'])
    data8 = data7.drop(['일자'],axis=1)
    data8 = data8.dropna(axis=0)
    
    df = sm.add_constant(data8, has_constant='add')

    x_list = list(df.columns.difference(['출고수량']))

    x = df[x_list]
    y = df['출고수량']

    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.00001,shuffle=True,random_state=44)

    Stepwise_best_model=Stepwise_model(X=x_train,y=y_train)
    
    # 예측할 범위의 데이터셋 생성
    ap_data = pd.DataFrame(columns=data6.columns)
    ap_data['일자'] = np.nan
    # 보유한 가장 마지막 데이터부터 5주
    start = data6.tail(1)['일자'].iloc[0] + pd.DateOffset(days=1)
    end = start+ pd.DateOffset(weeks=5)
    dates = pd.date_range(start,end,freq='W-Sat')
    ap_data['일자'] = dates
    ap_data['단가'] = data6.tail(1)['단가'].iloc[0]
    ap_data['주차'] = ap_data['일자'].dt.week
    ap_data['월'] = ap_data['일자'].dt.month
    ap_data['년'] = ap_data['일자'].dt.year
    ap_data1 = ap_data.copy()
    
    merge_data = pd.concat([data6,ap_data.head(1)])
    
    for i in range(len(ap_data1)):
        ap_data = ap_data[1:len(ap_data)]

        last_data = merge_data[['일자','출고수량']]

        abc = last_data[last_data['일자'] < merge_data['일자'].iloc[len(merge_data)-1]].tail(8)

        merge_data['8주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[0]
        merge_data['7주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[1]
        merge_data['6주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[2]
        merge_data['5주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[3]
        merge_data['4주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[4]
        merge_data['3주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[5]
        merge_data['2주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[6]
        merge_data['1주전출고량'].iloc[len(merge_data)-1] = abc['출고수량'].iloc[7]

        merge_data['4주평균'].iloc[len(merge_data)-1] = abc['출고수량'].tail(4).mean()
        merge_data['8주평균'].iloc[len(merge_data)-1] = abc['출고수량'].mean()
        merge_data['4주간최대'].iloc[len(merge_data)-1] = abc['출고수량'].tail(4).max()
        merge_data['4주간최소'].iloc[len(merge_data)-1]= abc['출고수량'].tail(4).min()
        
        merge_data['1주전변동'].iloc[len(merge_data)-1]= abc['출고수량'].iloc[6] - abc['출고수량'].iloc[7]
        merge_data['2주전변동'].iloc[len(merge_data)-1]= abc['출고수량'].iloc[5] - abc['출고수량'].iloc[6]
        merge_data['3주전변동'].iloc[len(merge_data)-1]= abc['출고수량'].iloc[4] - abc['출고수량'].iloc[5]
        merge_data['4주전변동'].iloc[len(merge_data)-1]= abc['출고수량'].iloc[3] - abc['출고수량'].iloc[4] 
        

        final_data = pd.get_dummies(merge_data,columns=['년','월','주차'])
        final_data = sm.add_constant(final_data, has_constant='add')
        final_data = final_data[Stepwise_best_model.model.exog_names]

        x_list= list(final_data.columns.difference(['출고수량']))
        final_x = final_data[x_list].iloc[len(final_data)-1]
        final = Stepwise_best_model.predict(final_x[Stepwise_best_model.model.exog_names])
        pred_y_stepwise = Stepwise_best_model.predict(x_test[Stepwise_best_model.model.exog_names])

        merge_data['출고수량'].iloc[len(merge_data)-1] = round(final.iloc[0],0)

        merge_data = pd.concat([merge_data,ap_data.head(1)])
    
    rsquared.append({item_list[n]:Stepwise_best_model.rsquared})
    rmse1.append({item_list[n]:np.sqrt(metrics.mean_squared_error(y_test, pred_y_stepwise))})

    result = merge_data.tail(len(ap_data1))
    result['품명'] = item_list[n]
    
    final_result.append(result) 


C:\Users\ioi\AppData\Local\Temp\ipykernel_13244\2459886093.py:63: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data6['주차'] = data6['일자'].dt.week
C:\Users\ioi\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Processed  86 models on 1 predictors in 0.1386549472808838
Selected predictors: ['1주전변동', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A2CB1B50>
forward
Processed  1 models on 0 predictors in 0.0019626617431640625
Selected predictors: ['const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A4568E50>
Processed  85 models on 2 predictors in 0.11170172691345215
Selected predictors: ['1주전변동', '년_2020', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A3FDB130>
forward
Processed  2 models on 1 predictors in 0.0049860477447509766
Selected predictors: ['1주전변동', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A436BE20>
Processed  84 models on 3 predictors in 0.10671448707580566
Selected predictors: ['1주전변동', '년_2020', '월_11', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper obje

Processed  71 models on 16 predictors in 0.09374833106994629
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가', '주차_17', '주차_4', '월_1', '월_12', '1주전출고량', '주차_51', '월_6', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A4634F40>
forward
Processed  16 models on 15 predictors in 0.021944046020507812
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가', '주차_4', '월_1', '월_12', '1주전출고량', '주차_51', '월_6', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A2B6B490>
Processed  70 models on 17 predictors in 0.10372018814086914
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가', '주차_17', '주차_4', '월_1', '월_12', '1주전출고량', '주차_51', '월_6', '주차_26', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A4568FA0>
forward

Processed  61 models on 26 predictors in 0.11170387268066406
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가', '주차_4', '월_1', '월_12', '1주전출고량', '주차_51', '월_6', '주차_26', '주차_9', '월_10', '주차_23', '주차_5', '년_2022', '8주평균', '주차_16', '주차_20', '년_2023', '주차_17', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A3FDB9D0>
forward
Processed  26 models on 25 predictors in 0.03690147399902344
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가', '주차_4', '월_1', '월_12', '1주전출고량', '주차_51', '월_6', '주차_26', '주차_9', '월_10', '주차_23', '주차_5', '년_2022', '8주평균', '주차_16', '주차_20', '년_2023', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A2B3F4C0>
backward
Processed  61 models on 26 predictors in 0.08876204490661621
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가

Processed  26 models on 25 predictors in 0.07280373573303223
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가', '주차_4', '월_1', '월_12', '1주전출고량', '주차_51', '월_6', '주차_26', '주차_9', '월_10', '주차_23', '주차_5', '년_2022', '8주평균', '주차_16', '주차_20', '년_2023', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208C8694BE0>
backward
Processed  61 models on 26 predictors in 0.17752671241760254
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가', '주차_4', '월_1', '월_12', '1주전출고량', '주차_51', '월_6', '주차_26', '주차_9', '월_10', '주차_23', '주차_5', '년_2022', '8주평균', '주차_16', '주차_20', '년_2023', '주차_17', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A4211370>
forward
Processed  26 models on 25 predictors in 0.10571861267089844
Selected predictors: ['년_2020', '월_11', '주차_42', '년_2021', '주차_30', '3주전변동', '6주전출고량', '주차_19', '단가

...

Processed  70 models on 17 predictors in 0.10571789741516113
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', '8주평균', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208C8780550>
forward
Processed  17 models on 16 predictors in 0.021941423416137695
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A28ADE50>
backward
Processed  70 models on 17 predictors in 0.12466573715209961
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', '8주평균', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper obj

Processed  70 models on 17 predictors in 0.1386268138885498
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', '8주평균', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208C8780490>
forward
Processed  17 models on 16 predictors in 0.034906864166259766
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A25CD1C0>
backward
Processed  70 models on 17 predictors in 0.10970616340637207
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', '8주평균', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper obje

Processed  70 models on 17 predictors in 0.10372376441955566
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', '8주평균', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A25CD340>
forward
Processed  17 models on 16 predictors in 0.06682133674621582
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A4734070>
backward
Processed  70 models on 17 predictors in 0.12067723274230957
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', '8주평균', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper obje

Processed  17 models on 16 predictors in 0.03490734100341797
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A28FB190>
backward
Processed  70 models on 17 predictors in 0.13962554931640625
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', '8주평균', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000208A4634CA0>
forward
Processed  17 models on 16 predictors in 0.035904884338378906
Selected predictors: ['4주간최소', '1주전출고량', '주차_43', '월_10', '주차_21', '주차_38', '주차_39', '7주전출고량', '주차_3', '4주전변동', '주차_31', '월_9', '주차_24', '주차_1', '주차_44', '주차_47', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0

C:\Users\ioi\AppData\Local\Temp\ipykernel_13244\2459886093.py:105: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  ap_data['주차'] = ap_data['일자'].dt.week
C:\Users\ioi\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\ioi\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\ioi\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\ioi\anacond

In [7]:
extra = []
for i in range(len(final_result)) :
               
    a = final_result[i]
    a = a.set_index('일자')
    b = a['출고수량'].resample('W-Sat').sum()
    b = b.reset_index()
    c = b[1:5]
    c = c.rename(columns = {'출고수량' : final_result[i]['품명'].iloc[0]})
    extra.append(c)
aaa = extra[0]
for i in range(len(extra)-1):
    aaa = pd.merge(aaa,extra[i+1])
aaa = aaa.astype({'일자':'string'})
a = aaa.copy()
a = a.set_index('일자')
b = np.transpose(a)
b['합'] = b.sum(axis=1)
b

일자,2023-03-04,2023-03-11,2023-03-18,2023-03-25,합
한촌탕고기(뒷덩이),107.0,78.0,81.0,79.0,345.0
한촌수육양지,139.0,124.0,148.0,145.0,556.0
한촌설렁탕육수(2.5kg),8582.0,8317.0,8602.0,8437.0,33938.0
삶은소건(2kg),101.0,104.0,104.0,104.0,413.0
사골추출액-1202(냉장),739.0,756.0,769.0,756.0,3020.0
한촌매운돼지갈비소스,59.0,67.0,61.0,65.0,252.0
한촌양념불고기,301.0,301.0,301.0,301.0,1204.0
혼합김치 18Kg,807.0,836.0,846.0,847.0,3336.0
곰탕육수(구.나주곰탕육수),162.0,163.0,167.0,170.0,662.0
"한촌수육머리고기(구,곰탕용)",265.0,288.0,295.0,315.0,1163.0
